# A brief summary of C++11
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents
* [Feature Introductions](#feature-introductions)
    * [Uniform Initialization](#uniform-initialization)
    * [The `auto` keyword](#the-auto-keyword)
    * [The `decltype` operator](#the-decltype-operator)  
    * [The `constexpr` keyword](#the-constexpr-keyword)
    * [The range-based `for` loop](#the-range-based-for-loop)
    * [R-value References](#r-value-references)
    * [Lambda Expressions](#lambda-expressions)
    * [User-defined Literals](#user-defined-literals)
* [Minor Ammendments](#minor-ammendments)
    * [Enumeration Classes](#enumeration-classes)
    * [The `nullptr` keyword](#the-nullptr-keyword)
    * [Raw String Literals](#raw-string-literals)
* [OOP Ammendments](#oop-ammendments)
    * [In-class Initializers](#in-class-initializers)
    * [`default` Methods](#default-methods)
    * [`delete` Functions](#delete-functions)
    * [Inheriting Constructors](#inheriting-constructors)
    * [Delegating Constructors](#delegating-constructors)
    * [The `override` keyword](#the-override-keyword)
    * [The `final` keyword](#the-final-keyword)
    * [`explicit` Conversion Operators](#explicit-conversion-operators)
* [Minor Emendations](#minor-emendations)
    * [`using` Type Aliases](#using-type-aliases)
    * [The `noexcept` keyword](#the-noexcept-keyword)
    * [`inline` Namespaces](#inline-namespaces)
* [Meta-programming](#meta-programming)
    * [Type Functions](#type-functions)
    * [Variadic Templates](#variadic-templates)

<hr>

### Feature Introductions <a class="anchor" id="feature-introductions"></a>

#### Uniform Initialization <a class="anchor" id="uniform-initialization"></a>

Direct initialization using parentheses, `()`, proved sometimes inconsistent. For example:

In [ ]:
int x();           // Interpreted as a function declaration

C++11 introduces a more consistent form of initialization, called *uniform* initialization, using braces, *{}*.

In [ ]:
int x{};           // Default-initialized to '0'

*Note:* Unlike `()`, `{}` does not allow implicit narrowing of values. For example, implicit conversion of `double` to `float` is prohibted.

Additionally, an immutable list, `initializer_list`, is defined in `<initializer_list>`.

If a constructor that accepts an `initializer_list` object exists, then uniform (or, brace) initialization allows for the initialization of the list implicitly by comma-separated values between the opening and closing braces.

This is similar in syntax to how C-style arrays are initialized.

In [559]:
//%cflags: -I.jupyter

#include <iostream>
#include <vector>
#include <initializer_list>
#include "helpers.h"

template <class T>
class Array {
    std::vector<T> v;
public:
    Array(const std::initializer_list<T> &init) : v(init.begin(), init.end()) {}
    Array(int size) : v(size) {}

    void show() { forwardly(v.begin(), v.end()); }
};

int main() {
    Array<int> arr{1, 2, 3};
    arr.show();
}

1, 2, 3,   


In [ ]:
Array<int> arr{1, 2, 3};               /* Equivalent syntax */

Array<int> arr = {1, 2, 3};

Array<int> arr({1, 2, 3});

In [ ]:
Array<int> arr(5);                     /* Array of size '5', default-initialized 'int's to '0' */

Array<int> arr{5};                     /* Array of size '1', consisting of {5} */

*Note:* When an `initializer_list` constructor is missing, `{}` behave similar to `()`.

#### The `auto` keyword <a class="anchor" id="the-auto-keyword"></a>

C++11 introduces the `auto` keyword, that may be used as a type, to be implicitly resolved at compile-time.

In [560]:
#include <iostream>

int main() {
    auto x = 5;
    
    std::cout << x;
}

5

In [561]:
#include <iostream>

auto add(int x, int y) {
    return x + y;
}

int main() {
    std::cout << add(1, 2);
}

3

*Note:* Always use copy initialization with the `auto` keyword. In particular, brace initialization resolves `auto` into `initializer_list`.

The `auto` keyword ignores references and top-level constants, but retains low-level constants.

In [562]:
#include <iostream>

int main() {
    const int y = 6;       /* 'y' is a top-level constant ('const int') */

    auto x = y;            /* auto resolved to 'int' */
    
    x = 7;                 /* Valid */
}      

In [ ]:
#include <iostream>

int main() {
    const int y = 6;       /* '&y' is a low-level constant ('const int *') */

    auto ptr = &y;         /* auto resolved to 'const int *' */
    
    *ptr = 7;              /* Error: Assignment of read-only location */
}      

*Note:* As an example, `const int` and `int * const` are top-level constants, while `const int *` is a low-level constant.

#### The `decltype` operator <a class="anchor" id="the-decltype-operator"></a>

C++11 introduces the `decltype` operator, that accepts an expression, and substitutes for its type at compile-time.

In [564]:
#include <iostream>

int main() {
    decltype(5 + 3.2) val = (5 + 3.2);
    
    std::cout << val;
}

8.2

Unlike the `auto` keyword, the `decltype` operator retains all qualifiers, including references and top-level constants.

In [ ]:
int main() {
    const int x = 5;          /* 'x' is a top-level const */
    
    decltype(x) y = 6;        /* Resolves to 'const int' */
    
    y = 7;                    /* Error: Assignment to read-only variable. */
}

#### The `constexpr` keyword <a class="anchor" id="the-constexpr-keyword"></a>

C++11 introduces the `constexpr` keyword, placed before a variable declaration gurantees that it is evaluated at compile-time.

When placed before a function, it gurantees that calls to this function are evaluated at compile-time.

In [569]:
#include <iostream>
                                      /* Before Compilation */
constexpr auto add(int x, int y) {
    return x + y;
}

int main() {
    constexpr int res = add(5, 6);
    
    std::cout << res;
}

11

In [567]:
#include <iostream>
                                      /* After Compilation */
int main() {
    std::cout << 11;
}

11

*Note:* One missing feature of `constexpr` functions, is the ability to overload them with non-`constexpr` functions.

A class with a `constexpr` constructor may be used as a literal.

In [570]:
#include <iostream>
                                      /* Before Compilation */
struct Point {
    int x, y;
    
    constexpr Point(int x, int y) : x(x), y(y) {}
};

int main() {
    constexpr Point p(1, 2);
    
    std::cout << "(" << p.x << ", " << p.y << ")";
}

(1, 2)

In [571]:
#include <iostream>
                                      /* After Compilation */
int main() {
    std::cout << "(" << 1 << ", " << 2 << ")";
}

(1, 2)

#### The range-based `for` loop <a class="anchor" id="the-range-based-for-loop"></a>

C++11 introduces a range-based `for` loop, that uses `begin` and `end` members of its post-colon argument.

In [572]:
#include <iostream>
#include <vector>

int main() {
    std::vector<int> v {1, 2, 3, 4, 5};

    for (auto &val : v) {
        std::cout << val << ' ';
    }
}

1 2 3 4 5 

*Note:* It is recommended to refrain from using a range-based `for` loop with a built-in array.

#### R-value References <a class="anchor" id="r-value-references"></a>

C++11 introduces another type of reference, an *rvalue reference*, as opposed to an *lvalue reference*.

In [ ]:
int x = 5;

int &l_ref1 = x;        /* Reference to an l-value */
int &l_ref2 = 6;        /* Error: Cannot assign r-value to l-value reference */

int &&r_ref = x;        /* Error: Cannot assign l-value to r-value reference */
int &&r_ref = 6;        /* Reference to an r-value */

const &cl_ref = x;      /* Constant reference to an l-value */
const &cl_ref = 6;      /* Constant reference to a temporary object, initialized with r-value */

A *move constructor* is a constructor that accepts an rvalue reference to an object of the same class.

It is meant to *steal* the internals of a temporary object, that uses dynamic memory, or other external resources that may be expensive to copy. Since the temporary object will not be used again, it saves us from the overhead of making a *deep-copy* of the temporary object (twice, once on the stack, and another inside the copy constructor), just to destroy it thereafter.

Additionally, an lvalue may be cast as an rvalue reference, using `static_cast`, and when passed to a move constructor, the lvalue is *moved* to another object, and is rendered useless thereafter.

In [575]:
#include <iostream>
#include <iterator>

class Integers {
    int *ptr;
    int size;
public:
    Integers(std::initializer_list<int> ls) {
        size = ls.size();
        ptr = new int(size);
        
        int i = 0;
        for (int item : ls) {
            ptr[i++] = item;
        }
    }
    
    Integers(Integers &&i) {           /* Move constructor */
        ptr = i.ptr;
        size = i.size;
        
        i.ptr = nullptr;
    }
    
    Integers(const Integers &ls) {     /* Copy constructor */
        size = ls.size;
        ptr = new int(size);
        
        int i = 0;
        for (int item : ls) {
            ptr[i++] = item;
        }
    }
    
    int * begin() { return ptr; }
    int * end() { return ptr + size; }
    
    const int * begin() const { return ptr; }
    const int * end() const { return ptr + size; }
};

int main() {
    Integers arr1 {1, 2, 3};
    Integers arr2 {arr1};                              /* Copy constructor (Deep-copy)    */
    Integers arr3 {static_cast<Integers &&>(arr1)};    /* Move constructor (Shallow-copy) */
    
    if (arr1.begin() == nullptr) { std::cout << "'arr1' has been moved.\n"; }
    if (arr2.begin() == nullptr) { std::cout << "'arr2' has been moved.\n"; }
    if (arr3.begin() == nullptr) { std::cout << "'arr3' has been moved.\n"; }
    
    return 0;
}

'arr1' has been moved.


*Note:* By default, a move constructor and a move assignment operator overloading function are defined.

*Note:* When a move constructor is defined, the implicit copy (and, move) constructor is no longer defined, and similarly for `=`.

*Note:* C++11 defines `move` function in `<utility>`, that returns an rvalue reference to an lvalue.

When returning an r-value or an r-value reference from a function that returns by value, the move constructor is called.

In [576]:
//%cflags: -O0

#include <iostream>
#include <utility>

struct A {
    A() { std::cout << "A()\n"; }
    A(A &&ref) { std::cout << "A(A &&)\n"; }
    A(const A &ref) { std::cout << "A(const A &)\n"; }
};

A getA() {
    A a;
    return std::move(a);
}

int main() {
    getA();
}

A()
A(A &&)


#### Lambda Expressions <a class="anchor" id="lambda-expressions"></a>

A *lambda expression* can be thought of as an anonymous function object.

In [577]:
#include <iostream>
#include <vector>
#include <functional>
                                  /* Implementation using function objects */
                                  /* Pros: Well-defined and re-usable */
template <class T>
class within_range {
    T lower, upper;
public:
    within_range(T lo, T up) : lower(lo), upper(up) {}
    
    bool operator() (T value) {
        return value >= lower && value <= upper;
    }
};

template<class F, class T>
class print_if {
    F fobj;
public:
    print_if(F fobj) : fobj(fobj) {}
    
    void operator() (T value) {
        if (fobj(value)) {
            std::cout << value << ' ';
        }
    }
};

template<class T>
auto m_within_range(T upper, T lower) {
    return within_range<T>(upper, lower);
}

template<class T, template<class> class F>
auto m_print_if(F<T> fobj) {
    return print_if<F<T>, T>(fobj);
}

int main() {
    std::vector<int> v{1, 2, 3, 4, 5, 6, 7, 8, 9};
    
    for_each(v.begin(), v.end(), m_print_if(m_within_range(3, 7)));
}

3 4 5 6 7 

In [578]:
#include <iostream>
#include <vector>
#include <functional>
                                  /* Implementation using lambda expressions */
                                  /* Pros: Short and concise. */

int main() {
    std::vector<int> v{1, 2, 3, 4, 5, 6, 7, 8, 9};
    
    for_each(v.begin(), v.end(), [] (int value) {
        if (value >= 3 && value <= 7) {
            std::cout << value << ' ';
        }
    });
}

3 4 5 6 7 

A lambda expression is composed of three parts:
* Capure list `[]`, which specifies the variables to be captured from the surrounding scope.
* Argument list `()`, which specifies the parameters passed upon call.
* Body `{}`, which contains the statements to be executed with each call.

`[]` is synonymous with the arguments passed to a function object's constructor, but it has its special syntax:

| *Capture List* | *Captures?* |
| :-- | :-- |
| `[]` | None. |
| `[var1, &var2]` | Copies local variable `var1` and references `var2`. |
| `[=, ...]` | Copies of all local variables, except which are specified next, are referenced. 
| `[&, ...]` | References to all local variables, except which are specified next, are copied. |
| `[this]` | References to all member variables. |

*Note:* `this` may be integrated into different capture lists, but the behavior may be inconsistent across compilers and future standards.

By default, the body of a lambda expression may not modify the captured variables, which is synonymous with defining `operator()` to be `const`. To modify the captured variables, `mutable` should preceed the body.

In [580]:
#include <iostream>

class Integer {
    int member;
public:
    Integer(int local) : member(local) {
        ([=, this] () mutable {
            local++;
            member++;
            return local;
        })();
        
        std::cout << "Local: " << local << '\n';
        std::cout << "Member: " << member << '\n';
    }
};

int main() {
    Integer(1);
}

Local: 1
Member: 2


To re-use a function object, allow the compiler to determine its type, by using the `auto` keyword.

In [581]:
#include <iostream>

class Integer {
    int member;
public:
    Integer(int local) : member(local) {
        auto lambda = ([=, this] () mutable {
            local++;
            member++;
            return local;
        });
        
        std::cout << "Lambda-Call: " << lambda() << '\n';
        std::cout << "Lambda-Call: " << lambda() << '\n';
        
        std::cout << "Local: " << local << '\n';
        std::cout << "Member: " << member << '\n';
    }
};

int main() {
    Integer(1);
}

Lambda-Call: 2
Lambda-Call: 3
Local: 1
Member: 3


Alternatively, and to be able to perform recursion, C++11 defines `std::function` in `<functional>`, that may be passed the return type and argument types of a lambda expression, and yield the proper type.

In [582]:
#include <iostream>
#include <functional>

int main() {
    std::function<void(int)> countdown = [&countdown] (int c) {
        std::cout << "Countdown: " << c << '\n';
        if (c > 0) { countdown(--c); }
    };
    
    countdown(3);
}

Countdown: 3
Countdown: 2


*Note:* The implementation of `std::function` is an application of meta-programming, and beyond the scope of this work.

By default, the return type of a lambda expression is determined implicitly. It may, however, be explicitly defined.

In [583]:
#include <iostream>

int main() {
    auto add = [] (double x, double y) -> int { 
        return x + y;
    };
    
    std::cout << add(4.6, 5.6);
}

10

In [ ]:
template<class T, class U>                 /* C++11 allows lambda return syntax, */
auto add(T x, U y) -> decltype(x*y);       /*  to be used in ordinary functions. */

*Note:* If `->` is unemployed, and no parameters are required, `()` may be omitted.

Global variables are always accessible within a lambda expression.

In [584]:
#include <iostream>

int cnt = 5;

int main() {
    auto inc = [] { return ++cnt; };
    
    for (int i = 0; i < 3; i++) {
        std::cout << inc();
    }
}

678

#### User-defined Literals <a class="anchor" id="user-defined-literals"></a>

C++11 introduces *user-defined literals* as a concept, applied by defining special suffixes to built-in literals.

In [610]:
#include <iostream>

struct Complex {
    double re, im;
    
    Complex(double re=0, double im=0) : re(re), im(im) {}
};

std::ostream & operator<< (std::ostream &os, const Complex &c) {
    os << "[Re]: " << c.re << ", [Im]: " << c.im << '\n';
    return os;
}

Complex operator"" _i(long double value) {
    return Complex(0, value);
}

Complex operator+ (const Complex &c1, const Complex &c2) {
    return Complex(c1.re + c2.re, c1.im + c2.im);
}

int main() {
    std::cout << Complex(1, 2);
    
    std::cout << 4.5_i;
    
    std::cout << 1 + 4.5_i;
}

[Re]: 1, [Im]: 2
[Re]: 0, [Im]: 4.5
[Re]: 1, [Im]: 4.5


There are four kinds of built-in literals, that can be suffixed to make a user-defined literal.

| Literal | Overloading Function |
| :--- | :--- |
| Integer | `CLASS operator"" _SUFFIX(unsigned long long)` |
| Floating | `CLASS operator"" _SUFFIX(long double)`  |
| String | `CLASS operator"" _SUFFIX(const char*, size_t)`  |
| Character | `CLASS operator"" _SUFFIX(char)`  |

*Note:* It is recommended to preceed user-defined literal suffixes with an underscore.

### Minor Ammendments <a class="anchor" id="minor-ammendments"></a>

#### Enumeration Classes <a class="anchor" id="enumeration-classes"></a>

C++11 introduces *enumeration classes*, as a scoped alternative to plain enumerations.

In [574]:
#include <iostream>

enum class Grade {High, Average, Low};

void tell_me(Grade grade) {
    switch (grade) {
        case Grade::High:
            std::cout << "He got a really high grade!";
            break;
        case Grade::Average:
            std::cout << "He's kind'a average.";
            break;
        case Grade::Low:
            std::cout << "He needs to pass the corrective exam, first!";
            break;
    }
}

int main() {
    tell_me(Grade::High);
}

He got a really high grade!

You may specify a specific integral type to contain values from the `enum class` type, which is `int` by default.

In [ ]:
enum class Grade : char {High, Average, Low};

*Note:* Implicit conversion from `int` to an `enum class` type, and vice versa, is prohibited.

#### The `nullptr` keyword <a class="anchor" id="the-nullptr-keyword"></a>

C++11 defines the `nullptr` keyword, to replace the ambigious zero-defined `NULL` macro.

In [573]:
//%cflags: -w

#include <iostream>

void foo(long int x) {
    std::cout << "foo(long int)\n";
}

void foo(long int *x) {
    std::cout << "foo(long int *)\n";
}

int main() {
    foo(NULL);
    foo(nullptr);
}

foo(long int)
foo(long int *)


*Note:* The statement `delete nullptr;` is harmless.

#### Raw String Literals <a class="anchor" id="raw-string-literals"></a>

C++11 introduces *raw string literals*, surrounded by `R"(` and `)"`, within which backslashes may not be escaped.

In [586]:
#include <iostream>

int main() {
    std::cout << R"(Hello.\n)";
}

Hello.\n

*Note:* Escape characters, such as `\n`, cannot be placed within a raw string literal.

As many *delimiting* characters may be placed between `"` and `(`, to distinguish the opening and closing from any similar sequence of characters within the raw string literal itself. It goes that the closing delimiting characters must match exactly.

In [587]:
#include <iostream>

int main() {
    std::cout << R"***(R"(Hello.\n)")***";
}

R"(Hello.\n)"

### OOP Ammendments <a class="anchor" id="oop-ammendments"></a>

#### In-class Initializers <a class="anchor" id="in-class-initializers"></a>

C++11 permits in-class initialization of member attributes.

In [592]:
#include <iostream>

int cnt = 1;

auto inc = [] (int &val) { return ++val; };
auto dec = [] (int &val) { return --val; };

struct A {
    int x = cnt;
    int y = inc(cnt);
    int z = dec(cnt);
    
    A() {}
    A(int x, int y, int z) : x(x), y(y), z(z) {}
};

std::ostream & operator<<(std::ostream &os, const A &a) {
    std::cout << a.x << ", " << a.y << ", " << a.z << '\n';
    return os;
}

int main() {
    std::cout << A();
    std::cout << A(7, 8, 9);
}

1, 2, 1
7, 8, 9


*Note:* If a constructor initializes a member, it overrides the default in-class initializer.

*Note:* An in-class initializer cannot use direct initialization, using `()`.

#### `default` Methods <a class="anchor" id="default-methods"></a>

C++11 allows the explicit definition of *default* methods, methods implictly defined by the compiler, and omitted only when overriden.

In [ ]:
class A {
    A() = default;                              /* Default constructor */
    
    A(const A &a) = default;                    /* Copy constructor    */
    
    A(A &&a) = default;                         /* Move constructor    */
    
    A & operator=(const A &a) = default;        /* Copy assignment     */
    
    A & operator=(A &&a) = default;             /* Move assignment     */
    
    ~A() = default;                             /* Default destructor  */
};

#### `delete` Functions <a class="anchor" id="delete-functions"></a>

C++11 allows for default methods to be explicitly *deleted* from a class.

In [ ]:
class A {
    A() = delete;
};

int main() {
    A();               /* Error: Use of deleted function. */
}

Another application of the `delete` keyword is to delete possible specializations of a template function.

In [ ]:
#include <iostream>
#include <string>

template<class T>
T add(T x, T y) { return x + y; }

std::string add(std::string, std::string) = delete;

int main() {
    std::cout << add(std::string("Hello,"),
                         std::string(" World."));          /* Error: Use of deleted function. */
                         
    std::cout << add<std::string>("Hello,", " World.");    /* Valid call. */
}

#### Inheriting Constructors <a class="anchor" id="inheriting-constructors"></a>

C++11 allows inheriting constructors from a base class.

In [615]:
//%cflags: -I.jupyter

#include <iostream>
#include <vector>
#include "helpers.h"

template<class T>
class MyVector : public std::vector<T> {
public:
    using std::vector<T>::vector;
};

int main() {
    MyVector<int> v{1, 2, 3, 4, 5};
    
    forwardly(v.begin(), v.end());
}

1, 2, 3, 4, 5,   


#### Delegating Constructors <a class="anchor" id="delegating-constructors"></a>

A *delegating constructor* is a constructor that delegates its operation to another constructor.

In [591]:
#include <iostream>

class A {
public:
    int a;
    
    A(int x) : a(x) {               /* Delegated-to */
        std::cout << "A(int)\n";
    }
    
    A() : A(10) {                   /* Delegating */
        std::cout << "A()\n";
    }
};

int main() {
    A();
}

A(int)
A()


*Note:* Construction delegation cannot be combined with a member initializer.

#### The `override` keyword <a class="anchor" id="the-override-keyword"></a>

C++11 introduces the `override` keyword, that may be placed before the body of a method in a derived class.

The compiler gurantees that any method marked as `override` must be overriding a `virtual` method in the base class.

In [ ]:
class A {
    virtual void a() {}
    
    void b() {}
};

class B : public A {
    void a() override {}         /* Valid. */
    
    void b() override {}         /* Error: 'B::b()' marked override, but does not override. */
};

#### The `final` keyword <a class="anchor" id="the-final-keyword"></a>

C++11 introduces the `final` keyword, that may be placed before the body of a method in a class.

The compiler gurantees that any `virtual` method marked as `final` cannot be overriden in a `virtual` class

In [ ]:
class A {
    virtual void foo() final {}
};

class B : public A {
    void foo() override {}        /* Error: virtual overriding final function */
};

#### `explicit` Conversion Operators <a class="anchor" id="explicit-conversion-operators"></a>

C++11 allows type conversion operator overloading methods to be declared `explicit`, a keyword previously reserved for constructors.

In [595]:
#include <iostream>

struct Integer {
    int value;
    
    Integer(int value) : value(value) {}
    
    explicit operator int () {
        return value;
    }
};

int main() {
    Integer x = 5;
    int y = static_cast<int>(x);
    
    std::cout << y;
}

5

### Minor Emendations <a class="anchor" id="minor-emendations"></a>

#### `using` Type Aliases <a class="anchor" id="using-type-aliases"></a>

C++11 allows the `using` keyword to define a type alias. 

In [ ]:
using Vector = std::vector<int>;         /* 'using' Type Alias */

In [ ]:
typedef std::vector<int> Vector;         /* 'typedef' Type Alias, equivalent */

The `using` keyword permits the definition of template aliases.

In [585]:
#include <iostream>
#include <utility>

template<class T>
using charAndValue = std::pair<char, T>;

int main() {
    charAndValue<int> p{'W', 1234};

    std::cout << p.first << '-' << p.second << '!';
}

W-1234!

#### The `noexcept` keyword <a class="anchor" id="the-noexcept-keyword"></a>

C++11 introduces the `noexcept` specification, equivalent to `throw ()`.

In [ ]:
int add(int, int) noexcept;

In [ ]:
int add(int, int) throw ();             /* Equivalent */

The `noexcept` specification may evaluate a `constexpr` expression, to determine specifically what the specification implies.

In [588]:
#include <iostream>

int one() noexcept(false) { return 1; }
int zero() noexcept(true) { return 0; }

int main() {
    std::cout << noexcept(one()) << noexcept(zero());
}

01

The `noexcept` operator returns a `bool`, indicating whether an expression may throw an exception.

It checks every function call within an expression, for the `noexcept` specification.

In [589]:
#include <iostream>
#include <vector>

int main() {
    std::cout << noexcept(std::vector<int>());    /* Zero-allocation, may not throw */
    std::cout << noexcept(std::vector<int>(1));   /* Allocates memory, may throw 'std::bad_alloc' */
}

10

#### `inline` Namespaces <a class="anchor" id="inline-namespaces"></a>

An *inline namespace* is, usually, a nested namespace, e.g: `B` nested within `A`, whose constituents may be accessed directly through the parent namespace, e.g: `B::`, or through normal, nested syntax, e.g: `B::A::`.

In [590]:
#include <iostream>
#include <string>

namespace AA {
    std::string secret = "Santa doesn't exist.";
}

namespace BB {
    std::string secret = "We didn't land on the moon.";
}

namespace Letter {
     inline namespace A {
         using namespace AA;
     }
     
     namespace B {
         using namespace BB;
     }
}

int main() {
    auto show = [] (std::string &s) { std::cout << s << '\n'; };

    show(AA::secret);
    show(BB::secret);
    show(Letter::secret);
    show(Letter::A::secret);
    show(Letter::B::secret);
}

Santa doesn't exist.
We didn't land on the moon.
Santa doesn't exist.
Santa doesn't exist.
We didn't land on the moon.


### Meta-programming <a class="anchor" id="meta-programming"></a>

*Meta-programming* is programming that manipulates programming entities. It usually employs templates to compute types and values at compile-time.

*Note:* The C++ meta-programming library is encapsulated in the `<type_traits>` header.

#### Type Functions <a class="anchor" id="type-functions"></a>

A *type* function is a function that either, takes at least one type argument, or produces at least one type as a result.

A *type predicate* is a type function that returns a `bool` value, `true` or `false`, based on the type argument(s). For example, `is_polymorphic` is a standard library type predicate, that returns `true` if the passed type is polymorphic, otherwise `false`. 

A *type selector* is a type function that selects between two, or more types based on a `bool` constant expression. For example, `conditional<BOOL, TYPE_A, TYPE_B>` is a standard library type selector, that yields `TYPE_A` if `BOOL` evaluates to `true`, otherwise `TYPE_B`.

In [645]:
#include <iostream>
#include <type_traits>

struct A {
    ~A() {}
};

struct B {
    virtual ~B() {}
};

int main() {
    typename std::conditional<std::is_polymorphic<A>::value, A, B>::type obj;

    std::cout << typeid(obj).name();
}

1B

In [648]:
template<bool BOOL, class TYPE_A, class TYPE_B>        /* Implementation of 'std::conditional' */
struct conditional {
    using type = TYPE_A;
};

template<class TYPE_A, class TYPE_B>
struct conditional<false, TYPE_A, TYPE_B> {
    using type = TYPE_B;
};

1B

*Note:* Type functions are templates in actuality, that receive arguments as template arguments, and return values or types as members.

A type alias can be used to simplify a call to type function that returns a type, allowing for the omission of the `typename` keyword.

In [646]:
#include <iostream>
#include <type_traits>

struct A {
    ~A() {}
};

struct B {
    virtual ~B() {}
};

template<class X, class Y>
using select_polymorphic = typename std::conditional<std::is_polymorphic<X>::value, X, Y>::type;

int main() {
    select_polymorphic<A, B> obj;

    std::cout << typeid(obj).name();
}

1B

#### Variadic Templates <a class="anchor" id="variadic-templates"></a>

A *variadic template* allows the passing of a variable number of template arguments. This becomes useful in multiple contexts.

For example, variadic templates enables the passing of a variable number of function arguments, each of a different type, called a *function argument pack*. Each argument has a corresponding type in the *template argument pack*.

In [690]:
#include <iostream>

template<class T>                  /* Terminating version, must preceed variadic version */
void printer(T arg) {
    std::cout << arg << '\n';
}

template<class T, class ... U>     /* Variadic version. */
void printer(T arg, U ... args) {
    std::cout << arg << '\n';
    printer(args ...);
}

int main() {
    printer(5, "Hello", 4.56f);

    return 0;
}

5
Hello
4.56


*Note:* The `std::tuple` variadic template class is a standard library generalization of `std::pair`.